In [713]:
# set up objects and necessary libs
import csv
import copy
from pprint import pprint as pprint
import statistics

participants = {}
blacklisted  = ['3291', '6734', '5072', '1161', '8035']

In [714]:
# read in the participants from the session
with open('csvs/roster.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    roster_count = 0
    for row in csv_reader:
        if row['participant'] not in blacklisted:
            participants[row['participant']] = {}
            roster_count += 1
            
print("Read in " + str(roster_count) + " participants from this session.")
#pprint(participants)

Read in 59 participants from this session.


In [715]:
# read the names of all of the participants
with open('csvs/COGS187_S219_Roster_7.31.19.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    names_count = 0
    for row in csv_reader:
        if (row['Code'] in participants):
            participants[row['Code']]['name'] = row['Student']
            names_count += 1
                
print("Found the names for " + str(names_count) + " out of " + str(roster_count) + " people from this session.")
#for participant in participants:
 #   print(participants[participant]['name'])

Found the names for 59 out of 59 people from this session.


In [716]:
# set up the participants object where each participant is mapped to each other participant
for participant in participants:
    for other_participant in participants:
        if participant != other_participant:
            participants[participant][other_participant] = [0,0]

#pprint(participants)

In [717]:
# get the interactions data from the app
with open('csvs/interactions_wnccx.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    roster_count = 0
    interactions = 0
    for row in csv_reader:
        if (row["people_interacted_with"][1:-1] != "blacklisted"):
            roster_count += 1
            worked_with = row["people_interacted_with"][1:-1].replace(";","").split(" ")
            worked_with_scores = row["all_interactions"][1:-1].split(" ")[1::2]
            for groupmate_num, groupmate in enumerate(worked_with):
                participants[row["participant"]][groupmate] = [1,float(worked_with_scores[groupmate_num][:-1])]
                interactions += 1
        line_count += 1
    print(f'Processed {line_count} lines.')
    print(f'Updated {roster_count} pids.')
    print(f'Tracked {interactions} interactions.')
    
#pprint(participants)

Processed 64 lines.
Updated 59 pids.
Tracked 692 interactions.


In [718]:
# get the demographic, social data
with open('csvs/coded_full_survey_data - full_survey_data.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    response_count = 0
    feature_count = 0
    invalid_response_count = 0
    for row in csv_reader:
        if (row['User_Code'] in participants):
            
            # demographic data
            participants[row['User_Code']]['age'] = row['Q7']
            participants[row['User_Code']]['gender'] = row['Q9']
            participants[row['User_Code']]['nationality'] = row['Q10']
            participants[row['User_Code']]['ethnicity'] = row['Q11']        
            if row['Q8'] == '1st year (Freshman)':
                participants[row['User_Code']]['school_year'] = 1
            elif row['Q8'] == '2nd year (Sophomore)':
                participants[row['User_Code']]['school_year'] = 2
            elif row['Q8'] == '3rd year (Junior)':
                participants[row['User_Code']]['school_year'] = 3
            elif row['Q8'] == '4th year (Senior)':
                participants[row['User_Code']]['school_year'] = 4
            elif row['Q8'] == '5th year or above':
                participants[row['User_Code']]['school_year'] = 5
            else:
                participants[row['User_Code']]['school_year'] = 6

            # personality factors
            participants[row['User_Code']]['agreeableness'] = row['SC10']
            participants[row['User_Code']]['conscientiousness'] = row['SC11']
            participants[row['User_Code']]['extraversion'] = row['SC8']
            participants[row['User_Code']]['emotional_stability'] = row['SC9']
            participants[row['User_Code']]['imagination'] = row['SC7']
           
            # individual characteristics
            participants[row['User_Code']]['psyc_collect'] = row['SC3']
            participants[row['User_Code']]['social_skills'] = row['SC4']
            participants[row['User_Code']]['leadership'] = row['SC5']
            participants[row['User_Code']]['creativity'] = row['SC1']
            participants[row['User_Code']]['intercult_sens'] = row['SC6']
            
            # 9 pillars 
            participants[row['User_Code']]['c_d'] = row['SC12']
            participants[row['User_Code']]['c_s'] = row['SC13']
            participants[row['User_Code']]['s_s'] = row['SC14']
            participants[row['User_Code']]['t_s'] = row['SC15']
            participants[row['User_Code']]['u_r'] = row['SC16']
            participants[row['User_Code']]['c_p'] = row['SC17']
            participants[row['User_Code']]['p_m'] = row['SC18']
            participants[row['User_Code']]['a_d'] = row['SC19']
            participants[row['User_Code']]['t_i'] = row['SC20']
       

            feature_count += 24
            response_count += 1
        else:
            invalid_response_count += 1
        line_count += 1
    print(f'Processed {line_count} lines.')
    print(f'Updated the demographic data for {response_count} people.')
    print(f'Produced {feature_count} features.')
    print(f'{invalid_response_count} people will not have their survey responses used.')

Processed 51 lines.
Updated the demographic data for 47 people.
Produced 1128 features.
4 people will not have their survey responses used.


In [719]:
#process the social_ties given by the survey results
nclose = 0
bclose = 0
mclose = 0
close = 0
vclose = 0

def getValueFromCloseness(closeness):
    global nclose, bclose, mclose, close, vclose
    if closeness == "I don't know them":
        nclose += 1
        return 0
    elif closeness == "I barely know them":
        bclose += 1
        return 1
    elif closeness == "We are moderately close":
        mclose += 1
        return 2
    elif closeness == "We are close":
        close += 1
        return 3
    elif closeness == "We are very close":
        vclose += 1
        return 4

with open('csvs/social_tie_responses.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    social_ties = 0
    waitlisted_count = 0
    waitlisted_people = []
    for row in csv_reader:
        # only check for people that are will be in the datasheet
        if row['Code'] in participants:
            curr_pid = row['Code']
            curr_name = participants[curr_pid]['name']
            if (curr_name not in row):
                    waitlisted_count += 1
                    waitlisted_people.append(curr_name)
            # get the ratings for the people they will have dyads with
            for participant in participants:
                name = participants[participant]['name']
                if participant != row['Code']:
                    if (name in row):
                        participants[curr_pid][participant].append(getValueFromCloseness(row[name]))
                        social_ties += 1
                    else:
                        # 0 rating for waitlisted people for now
                        participants[curr_pid][participant].append(0)

        line_count += 1
    print(f'Processed {line_count} lines.')
    print(f'Found and counted {nclose + bclose + mclose + close + vclose} out of {social_ties} social tie ratings.')
    print(f'{nclose} ties were considered not close.')
    print(f'{bclose} ties were considered barely close.')
    print(f'{mclose} ties were considered moderately close.')
    print(f'{close} ties were considered close.')
    print(f'{vclose} ties were considered very close.')
    print(f'Handled {waitlisted_count} people who were waitlisted. They are {waitlisted_people}')
    
#pprint(participants)
    

Processed 51 lines.
Found and counted 2400 out of 2400 social tie ratings.
2345 ties were considered not close.
39 ties were considered barely close.
14 ties were considered moderately close.
2 ties were considered close.
0 ties were considered very close.
Handled 3 people who were waitlisted. They are ['Jahadi, Kamran', 'Luxford, Steve', 'Ying, Zhuojun']


In [720]:
# mark down the last teams people were on
with open('csvs/last-teams_WNCCX.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    last_teammates = 0
    for row in csv_reader:
        if row['Participant'] in participants:
            for teammate in row['Last Team'].split(';'):
                if teammate in participants:
                    if 'last_team' not in participants[row['Participant']]:
                        participants[row['Participant']]['last_team'] = [teammate]
                    else:
                        participants[row['Participant']]['last_team'].append(teammate)
                    last_teammates += 1
        line_count += 1
        
    print(f'Processed {line_count} lines.')
    print(f'{last_teammates} last teammates were found.')
    
#pprint(participants)

Processed 62 lines.
120 last teammates were found.


In [721]:
# get the people who actually ended up teaming together 
with open('csvs/COGS 187A Final Teams - Form Responses 1.csv') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    team_count = 1
    final_teammates = 0
    for row in csv_reader:
        team_count += 1
        for i in range(1,7):
            curr_stud = row['Team Member '+ str(i) + ' - Name'] 
            if curr_stud in participants:
                for j in range(1, 7):
                    other_stud = row['Team Member '+ str(j) + ' - Name']
                    if j != i and other_stud in participants:
                        if 'final_team' not in participants[curr_stud]:
                            participants[curr_stud]['final_team'] = [other_stud]
                        else:
                            participants[curr_stud]['final_team'].append(other_stud)
                        final_teammates += 1
        line_count += 1
                        
    print(f'Processed {line_count} lines.')
    print(f'{final_teammates} final teammates were tracked.')
    
#pprint(participants)
            

Processed 10 lines.
246 final teammates were tracked.


In [722]:
# prune participants to only include people who participated in the field study and completed the survey
orig_participants = copy.deepcopy(participants)
features = ['name','age','gender','ethnicity','nationality','school_year','agreeableness',
            'conscientiousness','extraversion','emotional_stability','imagination','u_r',
            's_s','t_s','c_s','c_d','t_i','c_p','a_d','p_m','psyc_collect','social_skills',
            'leadership','creativity','intercult_sens','last_team', 'final_team']

for removee in orig_participants:
    # account for when we've already remove people
    if removee not in participants:
        continue
    for feature in features:
        # remove all info on this participant if they're missing a feature
        if feature not in participants[removee]:
            for keeper in participants:
                participants[keeper].pop(removee, None)
            participants.pop(removee, None)
            break
            
print(f'There are {len(participants)} valid data points left.')
pprint(participants.keys())
#pprint(participants)

There are 45 valid data points left.
dict_keys(['6769', '6801', '3936', '1537', '3596', '5345', '6055', '3535', '4224', '8227', '2431', '3314', '9182', '8039', '1982', '8617', '6077', '9085', '6550', '2255', '7475', '9674', '1805', '1801', '4111', '5835', '2766', '4826', '1290', '6067', '3322', '7379', '1790', '1828', '5116', '2534', '2524', '8137', '2337', '8369', '8966', '1222', '8009', '5304', '6910'])


In [723]:
# dyad maker and dyad tracker
class dyad:
    
    # construct the dyad opbject
    def __init__(self, personA, personB):
        
        # the name for this dyad
        self.dyad = personA + '-' + personB
        
        # calculated based on the average of how well the two people knew each other
        self.social_tie = ('1' if participants[personA][personB][2] > 0 or participants[personB][personA][2] > 0 else '0')
        self.mutual = ('1' if participants[personA][personB][2] > 0 and participants[personB][personA][2] > 0 else '0')
        self.social_tie_avg = str((participants[personA][personB][2] + participants[personB][personA][2]) / 2)

        # received from the interaction data
        self.interacted = ('0' if participants[personA][personB][0] == 0 else '1')
        self.avg_rating = str(participants[personA][personB][1])
        
        # these features come from the pre-survey
        
        # The value for these feature is either 0/1 for same/different or the difference between the two people
        self.age = str(abs(int(participants[personA]['age']) - int(participants[personB]['age'])))
        self.gender = ('1' if participants[personA]['gender'] == participants[personB]['gender'] else '0')
        self.both_male = ('1' if participants[personA]['gender'] == 'Man' 
                              and participants[personB]['gender'] == 'Man' else '0')
        self.both_female = ('1' if participants[personA]['gender'] == 'Woman' 
                              and participants[personB]['gender'] == 'Woman' else '0')
        self.both_non_binary = ('1' if participants[personA]['gender'] == 'Non-binary' 
                              and participants[personB]['gender'] == 'Non-binary' else '0')
        self.ethnicity = ('1' if participants[personA]['ethnicity'] == participants[personB]['ethnicity'] else '0')
        self.nationality = ('1' if participants[personA]['nationality'] == 
                                    participants[personB]['nationality'] else '0')
        self.school_year = str(abs(int(participants[personA]['school_year']) - 
                                   int(participants[personB]['school_year'])))
        #self.same_year = ''
        
        # personality factors, value is the difference between each person
        self.agreeableness = str(abs(int(participants[personA]['agreeableness']) - 
                                     int(participants[personB]['agreeableness'])))
        self.conscientiousness = str(abs(int(participants[personA]['conscientiousness']) - 
                                     int(participants[personB]['conscientiousness'])))
        self.extraversion = str(abs(int(participants[personA]['extraversion']) - 
                                     int(participants[personB]['extraversion']))) 
        self.emot_stab = str(abs(int(participants[personA]['emotional_stability']) - 
                                     int(participants[personB]['emotional_stability'])))
        self.imagination = str(abs(int(participants[personA]['imagination']) - 
                                     int(participants[personB]['imagination'])))
        
        # 9 pillars, value is the difference between each person
        self.u_r = str(abs(int(participants[personA]['u_r']) - int(participants[personB]['u_r'])))
        self.s_s = str(abs(int(participants[personA]['s_s']) - int(participants[personB]['s_s'])))
        self.t_s = str(abs(int(participants[personA]['t_s']) - int(participants[personB]['t_s'])))
        self.c_s = str(abs(int(participants[personA]['c_s']) - int(participants[personB]['c_s'])))
        self.a_d = str(abs(int(participants[personA]['a_d']) - int(participants[personB]['a_d'])))
        self.t_i = str(abs(int(participants[personA]['t_i']) - int(participants[personB]['t_i'])))
        self.c_p = str(abs(int(participants[personA]['c_p']) - int(participants[personB]['c_p'])))
        self.c_d = str(abs(int(participants[personA]['c_d']) - int(participants[personB]['c_d'])))
        self.p_m = str(abs(int(participants[personA]['p_m']) - int(participants[personB]['p_m'])))
        
        # individual characteristics, value is the difference between each person
        self.psyc_collect = str(abs(int(participants[personA]['psyc_collect']) - 
                                     int(participants[personB]['psyc_collect'])))
        self.social_skills = str(abs(int(participants[personA]['social_skills']) - 
                                     int(participants[personB]['social_skills'])))
        self.leadership = str(abs(int(participants[personA]['leadership']) - 
                                     int(participants[personB]['leadership'])))
        self.creativity = str(abs(int(participants[personA]['creativity']) - 
                                     int(participants[personB]['creativity'])))
        self.intercult_sens = str(abs(int(participants[personA]['intercult_sens']) - 
                                     int(participants[personB]['intercult_sens'])))
        
         # 0/1 depending on whether these two people were on their last team together
        self.last_team = ('1' if personB in participants[personA]['last_team'] else '0')
        
        # 0/1 depending on whether or not these two ended up on the same team, aka, the truth-value for our dyad
        self.team = ('1' if personB in participants[personA]['final_team'] else '0')
        
    # turn our dyad object into a string
    def toString(self):
        return ({'dyad': self.dyad,'social_tie': self.social_tie,'mutual': self.mutual,
                 'social_tie_avg': self.social_tie_avg,'interacted': self.interacted,'avg_rating': self.avg_rating,
                 'age': self.age, 'gender': self.gender, 'both_male': self.both_male,
                 'both_female': self.both_female, 'both_non_binary': self.both_non_binary, 
                 'ethnicity': self.ethnicity, 'nationality': self.nationality,'school_year': self.school_year,
                 'agreeableness': self.agreeableness,'conscientiousness': self.conscientiousness, 
                 'extraversion': self.extraversion,'emotional_stability': self.emot_stab, 
                 'imagination': self.imagination, 'u_r': self.u_r,'s_s': self.s_s, 't_s': self.t_s, 'c_s': self.c_s,
                 'a_d': self.a_d, 't_i': self.t_i,'c_p': self.c_p, 'c_d': self.c_d, 'p_m': self.p_m, 
                 'psychological_collectivism': self.psyc_collect,'social_skills': self.social_skills, 
                 'leadership': self.leadership, 'creativity': self.creativity,
                 'intercultural_sensitivity': self.intercult_sens, 'last_team': self.last_team, 'team': self.team})

In [724]:
# build the master spreadsheet of dyad data
header = ['dyad','social_tie','mutual','social_tie_avg','interacted','avg_rating']
header += ['age','gender','both_male','both_female','both_non_binary','ethnicity','nationality','school_year']
header += ['agreeableness','conscientiousness','extraversion','emotional_stability','imagination']
header += ['u_r','s_s','t_s','c_s','a_d','t_i','c_p','c_d','p_m']  
header += ['psychological_collectivism','social_skills','leadership','creativity','intercultural_sensitivity']
header += ['last_team','team']

dyad_count = 0
dyads = {}

with open('csvs/dyad_data_master.csv', mode='w') as dyad_file:
    
    # set up the spreadsheet to be ready for writing and with it's header
    dyad_writer = csv.DictWriter(dyad_file, fieldnames=header)
    dyad_writer.writeheader()
    
    # produce each dyad
    for participant in participants:
        for other_participant in participants[participant]:
             if (participant != other_participant and 
                 (other_participant + '-' + participant) not in dyads and
                 other_participant not in features):
                new_dyad = dyad(participant, other_participant)
                dyad_writer.writerow(new_dyad.toString())
                dyads[new_dyad.dyad] = new_dyad
                dyad_count += 1

#pprint(dyads)    
print(f'Completed {dyad_count} dyads.')

Completed 990 dyads.


In [725]:
# find some percentages of teams formed based on social_tie, interactions, etc.
interacted_dyad_count = 0
teamed_dyad_count = 0
for dyad in dyads:
    if (dyads[dyad].interacted == '1'):
        interacted_dyad_count += 1
        if (dyads[dyad].team == '1'):
            teamed_dyad_count += 1
            
           

print(f'{interacted_dyad_count} Prototeams interactions were made.')
print(f'{teamed_dyad_count} final teammates who interacted during Prototeams were found.')
print(f'{round((teamed_dyad_count / interacted_dyad_count) * 100, 3)}% of Prototeams interactions ended up in a team.\n')


teamed_dyad_count = 0
st_dyad_count = 0
for dyad in dyads:
    if (dyads[dyad].social_tie == '1'):
        st_dyad_count += 1
        if (dyads[dyad].team == '1'):
            teamed_dyad_count += 1

            
print(f'{st_dyad_count} previous social ties were found.')           
print(f'{teamed_dyad_count} teammates who had previous social ties were found.')
print(f'{round((teamed_dyad_count / st_dyad_count) * 100, 3)}% of social ties ended up in a team.\n') 

interacted_dyad_count = 0
st_dyad_count = 0
teamed_dyad_count = 0
for dyad in dyads:
    if (dyads[dyad].team == '1'):
        teamed_dyad_count += 1
        if (dyads[dyad].interacted == '1'):
            interacted_dyad_count += 1
        if (dyads[dyad].social_tie == '1'):
            st_dyad_count += 1

            
print(f'{teamed_dyad_count} teammates were found.')           
print(f'{interacted_dyad_count} Prototeam interactions ended up in final teams were found.')
print(f'{st_dyad_count} social ties ended up in final teams.')
print(f'{round((interacted_dyad_count / teamed_dyad_count) * 100, 3)}% of occured teams interacted during the activity.')
print(f'{round((st_dyad_count / teamed_dyad_count) * 100, 3)}% of occured teammates had previous social ties.\n')
         

202 Prototeams interactions were made.
26 final teammates who interacted during Prototeams were found.
12.871% of Prototeams interactions ended up in a team.

36 previous social ties were found.
15 teammates who had previous social ties were found.
41.667% of social ties ended up in a team.

86 teammates were found.
26 Prototeam interactions ended up in final teams were found.
15 social ties ended up in final teams.
30.233% of occured teams interacted during the activity.
17.442% of occured teammates had previous social ties.



In [726]:
# who to interview?
with open('csvs/COGS187_S219_Roster_7.31.19.csv', mode='r') as csv_file:
    roster_reader = csv.DictReader(csv_file)
    with open('csvs/who_to_email.csv', mode='w') as csv_file2:
        roster_writer = csv.DictWriter(csv_file2, fieldnames=['Interview'])                                    
        roster_writer.writeheader()

        for row in roster_reader:
            if row['Code'] in participants:
                roster_writer.writerow({"Interview": "TRUE"})
            else:
                roster_writer.writerow({"Interview": "FALSE"})



In [727]:
# diversity metrics
def find_gender_diversity(members):
    categories = {'Man': 0, 'Woman': 0, 'Non-binary': 0}
    num_mems = len(members)
    for member in members:
        if participants[member]['gender'] == 'Man':
            categories['Man'] += 1
        if participants[member]['gender'] == 'Woman':
            categories['Woman'] += 1
        if participants[member]['gender'] == 'Non-binary':
            categories['Non-binary'] += 1
    # blau's index
    gender_sum = 0
    for category in categories:
        gender_sum += (categories[category]*(categories[category]))/(num_mems*(num_mems-1))
    #print(1 - gender_sum)
    # just return the counts
    return categories['Man'], categories['Woman'], categories['Non-binary']


In [728]:
# build a team facts datasheet
with open('csvs/COGS 187A Final Teams - Form Responses 1.csv') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    team_count = 0
    proto_count = 0
    people_count = 0
    fieldnames = ['Team','Members','PT_Participants','Num_ProtoTeams_Interactions','Interacted',
                  'Num_Social_Ties','Social_Ties','Average_Social_Tie_Rating',
                  'Age_Variance','Num_Male','Num_Female','Num_Non_Binary','Avg_Social_Skills']
    # col for prior interactions Score! age spread, social skills
    with open('csvs/teams_info.csv', mode='w') as csv_file2:
        teams_writer = csv.DictWriter(csv_file2, fieldnames=fieldnames)                                    
        teams_writer.writeheader()
        
              
        # build a age-to-team dataset
        with open('csvs/ages_and_teams.csv', mode='w') as csv_file3:
            age_writer = csv.DictWriter(csv_file3, fieldnames=['Person','Age','Team'])                                    
            age_writer.writeheader()

            for row in csv_reader:
                # get the members of this team
                team_count += 1
                curr_team = row['What is your team name?']
                members = []
                proto_members = []
                for i in range(1,7):
                    curr_stud = row['Team Member '+ str(i) + ' - Name'] 
                    members.append(curr_stud)
                    people_count += 1
                    if curr_stud in participants:
                        proto_count += 1
                        proto_members.append(curr_stud)
                        
                # things we are interested in        
                interactions_sum = 0
                interacted_people = []
                prior_social_ties = 0
                prior_people = []
                avg_prior_rating = 0
                ages = []
                social_skill_scores = []
                
                # loop through the team members to get some important info
                for person in proto_members:
                    person_age = int(participants[person]['age'])
                    age_writer.writerow({'Person': person, 'Age': person_age, 'Team': curr_team})
                    ages.append(person_age)
                    social_skill_scores.append(int(participants[person]['social_skills']))
                    for other_person in proto_members:
                        if (person + '-' + other_person) in dyads:
                            curr_dyad = dyads[(person + '-' + other_person)]
                            if int(curr_dyad.interacted) == 1:
                                interactions_sum += 1
                                interacted_people += [[person,other_person]]
                            if int(curr_dyad.social_tie) == 1:
                                prior_social_ties += 1
                                prior_people += [[person,other_person]]
                                avg_prior_rating += float(curr_dyad.social_tie_avg)
                    
                if (prior_social_ties > 0):
                    avg_prior_rating = avg_prior_rating / prior_social_ties 
                    
                # number of men, women, and non-binary
                man_count, woman_count, nb_count = find_gender_diversity(proto_members)
                
                # coefficient of variance for ages
                age_var = round(statistics.stdev(ages) / statistics.mean(ages), 3)
                
                # avg social skills score
                ss_avg = round(statistics.mean(social_skill_scores), 3)
                
                # write our to team info csv file
                teams_writer.writerow({'Team': curr_team, 'Members': members, 'PT_Participants': proto_members,
                                       'Num_ProtoTeams_Interactions': interactions_sum, 'Interacted': interacted_people,
                                       'Num_Social_Ties': prior_social_ties, 'Social_Ties': prior_people,
                                       'Average_Social_Tie_Rating': avg_prior_rating, 'Age_Variance': age_var,
                                       'Num_Male': man_count,'Num_Female': woman_count, 'Num_Non_Binary': nb_count,
                                       'Avg_Social_Skills': ss_avg})


    print(f'{team_count} teams were tracked.')
    print(f'There are {people_count} people overall.')
    print(f'{proto_count} of them participated in ProtoTeams.')
    
#pprint(participants)
            

10 teams were tracked.
There are 60 people overall.
45 of them participated in ProtoTeams.
